# Data Prerprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/kkp_info.csv')
df.head()

,Unnamed: 0,id,Title,Writer,Readers,Total Comments,Genre,Grade,Days of Week,Total Episodes,Description
0,3,58289261,남주들의 집착보다 내 탈영이 빠르겠다,아듀,29.7만명,1.4만,기다무소설로판,전체이용가,"월, 화, 수, 토, 일 연재 | 독점",97,"""너는 오래 전 죽은 그 애를 참 닮았군.""의 '그 애' 역할로 빙의했다.\n\n군..."
1,4,55275635,마법황가 무공천재,주문향,31.7만명,"7,712",기다무소설판타지,전체이용가,완결,298,[카카오 독점 연재]\n\n혈마(血魔) 천무정.\n피도 눈물도 없는 무림 최강의 존...
2,5,52716578,불패검선,적하,30.8만명,"6,674",기다무소설무협,전체이용가,완결,253,"시골 객점의 아들 송우문.\n어릴 적부터 몸이 약했던 그는,\n어느 날 노도사가 그..."
3,6,58404017,7번의 결혼이 예정되어 있습니다,커피바닐라,32.3만명,"5,112",기다무소설로판,전체이용가,"월, 화, 수, 목, 토, 일 연재 | 독점",94,아가사 노빌리스는 몇 세기 만에 나온 어마어마한 자산의 상속녀였다.\n\n결혼을 6...
4,7,47050646,템빨,박새날,281.8만명,54.9만,기다무소설판타지,전체이용가,"월, 수, 금, 토 연재",1581,공사장에서 벽돌 나르고 삽질하는 불운한 인생의 신영우. 그런데 심지어 게임 속에서 ...


In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df.columns = ['id', 'title', 'writer', 'readers', 'total_comments', 'genre', 'grade', 'days_of_week', 'total_episode', 'description']
df.head()

,id,title,writer,readers,total_comments,genre,grade,days_of_week,total_episode,description
0,58289261,남주들의 집착보다 내 탈영이 빠르겠다,아듀,29.7만명,1.4만,기다무소설로판,전체이용가,"월, 화, 수, 토, 일 연재 | 독점",97,"""너는 오래 전 죽은 그 애를 참 닮았군.""의 '그 애' 역할로 빙의했다.\n\n군..."
1,55275635,마법황가 무공천재,주문향,31.7만명,"7,712",기다무소설판타지,전체이용가,완결,298,[카카오 독점 연재]\n\n혈마(血魔) 천무정.\n피도 눈물도 없는 무림 최강의 존...
2,52716578,불패검선,적하,30.8만명,"6,674",기다무소설무협,전체이용가,완결,253,"시골 객점의 아들 송우문.\n어릴 적부터 몸이 약했던 그는,\n어느 날 노도사가 그..."
3,58404017,7번의 결혼이 예정되어 있습니다,커피바닐라,32.3만명,"5,112",기다무소설로판,전체이용가,"월, 화, 수, 목, 토, 일 연재 | 독점",94,아가사 노빌리스는 몇 세기 만에 나온 어마어마한 자산의 상속녀였다.\n\n결혼을 6...
4,47050646,템빨,박새날,281.8만명,54.9만,기다무소설판타지,전체이용가,"월, 수, 금, 토 연재",1581,공사장에서 벽돌 나르고 삽질하는 불운한 인생의 신영우. 그런데 심지어 게임 속에서 ...


In [ ]:
df['free'] = df.genre.str.split('소설').str[0]
df['genre'] = df.genre.str.split('소설').str[1]

df['days_of_week'] = df['days_of_week'].apply(lambda x: x.replace(' ', ''))
df['kkp_only'] = df.days_of_week.str.split('연재').str[1]
df['kkp_only'] = df['kkp_only'].apply(lambda x: 1 if x=='|독점' in x else 0)
df['days_of_week'] = df.days_of_week.str.split('연재').str[0]
df.head()

,id,title,writer,readers,total_comments,genre,grade,days_of_week,total_episode,description,free,kkp_only
0,58289261,남주들의 집착보다 내 탈영이 빠르겠다,아듀,29.7만명,1.4만,로판,전체이용가,"월,화,수,토,일",97,"""너는 오래 전 죽은 그 애를 참 닮았군.""의 '그 애' 역할로 빙의했다.\n\n군...",기다무,1
1,55275635,마법황가 무공천재,주문향,31.7만명,"7,712",판타지,전체이용가,완결,298,[카카오 독점 연재]\n\n혈마(血魔) 천무정.\n피도 눈물도 없는 무림 최강의 존...,기다무,0
2,52716578,불패검선,적하,30.8만명,"6,674",무협,전체이용가,완결,253,"시골 객점의 아들 송우문.\n어릴 적부터 몸이 약했던 그는,\n어느 날 노도사가 그...",기다무,0
3,58404017,7번의 결혼이 예정되어 있습니다,커피바닐라,32.3만명,"5,112",로판,전체이용가,"월,화,수,목,토,일",94,아가사 노빌리스는 몇 세기 만에 나온 어마어마한 자산의 상속녀였다.\n\n결혼을 6...,기다무,1
4,47050646,템빨,박새날,281.8만명,54.9만,판타지,전체이용가,"월,수,금,토",1581,공사장에서 벽돌 나르고 삽질하는 불운한 인생의 신영우. 그런데 심지어 게임 속에서 ...,기다무,0


In [ ]:
df['total_comments'] = df['total_comments'].apply(lambda x: x.replace('만', '000') if ('만' in x) and ('.' in x) else x)
df['total_comments'] = df['total_comments'].apply(lambda x: x.replace('만', '0000') if ('만' in x) else x)
df['total_comments'] = df['total_comments'].apply(lambda x: x.replace('.', '') if ('.' in x) else x)
df['total_comments'] = df['total_comments'].apply(lambda x: x.replace(',', '') if (',' in x) else x)
df['total_comments'] = pd.to_numeric(df['total_comments'])

df['readers'] = df['readers'].apply(lambda x: x.replace('명', ''))
df['readers'] = df['readers'].apply(lambda x: x.replace('만', '000') if ('만' in x) and ('.' in x) else x)
df['readers'] = df['readers'].apply(lambda x: x.replace('만', '0000') if ('만' in x) else x)
df['readers'] = df['readers'].apply(lambda x: x.replace('.', '') if ('.' in x) else x)
df['readers'] = df['readers'].apply(lambda x: x.replace(',', '') if (',' in x) else x)
df['readers'] = pd.to_numeric(df['readers'])

df.head()

,id,title,writer,readers,total_comments,genre,grade,days_of_week,total_episode,description,free,kkp_only
0,58289261,남주들의 집착보다 내 탈영이 빠르겠다,아듀,297000,14000,로판,전체이용가,"월,화,수,토,일",97,"""너는 오래 전 죽은 그 애를 참 닮았군.""의 '그 애' 역할로 빙의했다.\n\n군...",기다무,1
1,55275635,마법황가 무공천재,주문향,317000,7712,판타지,전체이용가,완결,298,[카카오 독점 연재]\n\n혈마(血魔) 천무정.\n피도 눈물도 없는 무림 최강의 존...,기다무,0
2,52716578,불패검선,적하,308000,6674,무협,전체이용가,완결,253,"시골 객점의 아들 송우문.\n어릴 적부터 몸이 약했던 그는,\n어느 날 노도사가 그...",기다무,0
3,58404017,7번의 결혼이 예정되어 있습니다,커피바닐라,323000,5112,로판,전체이용가,"월,화,수,목,토,일",94,아가사 노빌리스는 몇 세기 만에 나온 어마어마한 자산의 상속녀였다.\n\n결혼을 6...,기다무,1
4,47050646,템빨,박새날,2818000,549000,판타지,전체이용가,"월,수,금,토",1581,공사장에서 벽돌 나르고 삽질하는 불운한 인생의 신영우. 그런데 심지어 게임 속에서 ...,기다무,0


### Description
> 수집된 데이터는 2021.12.14 기준입니다.

- id : 고유 ID
- title : 제목
- writer : 작가명
- total_comments : 총 댓글 수
- genre : 장르
- grade : 이용 등급
- days_of_week : 연재 요일 혹은 완결 여부
- total_episodes : 총 연재 편 수
- description : 작품 소개
- free : 기다리면 무료 여부
- kkp_only : 카카오 페이지 독점 여부


# 자연어 처리용 데이터

In [ ]:
df_nlp = df[['description', 'title', 'genre']]
df_nlp.head()

,description,title,genre
0,"""너는 오래 전 죽은 그 애를 참 닮았군.""의 '그 애' 역할로 빙의했다.\n\n군...",남주들의 집착보다 내 탈영이 빠르겠다,로판
1,[카카오 독점 연재]\n\n혈마(血魔) 천무정.\n피도 눈물도 없는 무림 최강의 존...,마법황가 무공천재,판타지
2,"시골 객점의 아들 송우문.\n어릴 적부터 몸이 약했던 그는,\n어느 날 노도사가 그...",불패검선,무협
3,아가사 노빌리스는 몇 세기 만에 나온 어마어마한 자산의 상속녀였다.\n\n결혼을 6...,7번의 결혼이 예정되어 있습니다,로판
4,공사장에서 벽돌 나르고 삽질하는 불운한 인생의 신영우. 그런데 심지어 게임 속에서 ...,템빨,판타지


In [ ]:
df_nlp['description'] = df_nlp['description'].apply(lambda x: x.replace('\n', ' '))
df_nlp.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,description,title,genre
0,"""너는 오래 전 죽은 그 애를 참 닮았군.""의 '그 애' 역할로 빙의했다. 군부를...",남주들의 집착보다 내 탈영이 빠르겠다,로판
1,[카카오 독점 연재] 혈마(血魔) 천무정. 피도 눈물도 없는 무림 최강의 존재. ...,마법황가 무공천재,판타지
2,"시골 객점의 아들 송우문. 어릴 적부터 몸이 약했던 그는, 어느 날 노도사가 그려 ...",불패검선,무협
3,아가사 노빌리스는 몇 세기 만에 나온 어마어마한 자산의 상속녀였다. 결혼을 6번 ...,7번의 결혼이 예정되어 있습니다,로판
4,공사장에서 벽돌 나르고 삽질하는 불운한 인생의 신영우. 그런데 심지어 게임 속에서 ...,템빨,판타지


In [ ]:
# tsv로 저장
df_nlp.to_csv('/content/drive/MyDrive/description.tsv', sep='\t')